# DeviaMetric - Calculateur de RMSD 

Bienvenue dans DeviaMetric ! Cet outil est conçu pour calculer le RMSD (Root Mean Square Deviation) entre deux fichiers PDB, tout en offrant une visualisation interactive de l'alignement entre les structures moléculaires correspondantes.

---

## À propos de DeviaMetric 🌟

Le RMSD est une mesure couramment utilisée en bio-informatique pour évaluer la similarité structurelle entre deux ensembles d'atomes. Plus précisément, cette méthode calcule le RMSD entre deux structures alignées en fonction du type d'alignement spécifié, tel que les carbones alpha, tous les atomes, ou seulement le backbone. Les atomes des deux structures sont comparés atom par atom, et le RMSD est calculé en prenant en compte la différence quadratique moyenne entre les positions des atomes, après une superposition optimale.

DeviaMetric propose une interface conviviale permettant de charger deux fichiers PDB, de choisir les atomes sur lesquels baser l'alignement (par exemple, tous les atomes, uniquement le squelette de la protéine, ou les atomes carbone-alpha), de visualiser l'alignement en 3D à l'aide de NGLView, et enfin de calculer le RMSD correspondant.

---

## Configuration Requise 🔧

Avant de commencer, assurez-vous d'avoir installé les dépendances suivantes :
- [Biopython](https://biopython.org/)
- [NGLView](https://github.com/nglviewer/nglview)
- [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)


---

**Remarque :** Ce notebook est interactif et nécessite une exécution cellule par cellule pour afficher les résultats.


## I. Chargement des librairies

In [1]:
import nglview as nv
import numpy as np
from Bio import BiopythonWarning
from Bio.PDB import PDBParser, Superimposer
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual
from IPython.display import display, HTML 
import ipywidgets as widgets
import warnings
import urllib.request
import os

In [2]:
# pour ignorer les warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.simplefilter('ignore', BiopythonWarning)

## II. Chargement des structures

Par recherche dans la [RCSB PDB](https://www.rcsb.org/) :

In [4]:
# Créer les widgets pour les deux ID PDB
pdb_id_1 = widgets.Text(
    value='',
    placeholder='Entrez le premier ID PDB',
    description='ID PDB 1:',
    disabled=False
)

pdb_id_2 = widgets.Text(
    value='',
    placeholder='Entrez le deuxième ID PDB',
    description='ID PDB 2:',
    disabled=False
)

# Définir la fonction pour afficher les structures PDB
def visualisation_par_id(pdb_id_1, pdb_id_2):
    pdb_id1 = pdb_id_1.lower()
    pdb_id2 = pdb_id_2.lower()
    if pdb_id1 != '' and pdb_id2 != '':
        if len(pdb_id1) == 4 and len(pdb_id2) == 4:
            view1 = nv.show_pdbid(pdb_id1)
            view2 = nv.show_pdbid(pdb_id2)

            view1.add_label(pdb_id1, font_size=24)
            view2.add_label(pdb_id2, font_size=24)
            
            display(view1)
            display(view2)
        else:
            print("Les ID PDB ne sont pas valides")
    else:
        print('Veuillez entrer les deux ID PDB')

# Lier la fonction visualisation aux widgets et créer une interface interactive
widgets.interact(visualisation_par_id, pdb_id_1=pdb_id_1, pdb_id_2=pdb_id_2);
print("Attendez quelques secondes pour que les structures apparaissent...")

interactive(children=(Text(value='', description='ID PDB 1:', placeholder='Entrez le premier ID PDB'), Text(va…

Attendez quelques secondes pour que les structures apparaissent...


In [5]:
def get_structure_from_pdb_id(pdb_id):
    print(f"Téléchargement du fichier PDB de l'ID {pdb_id} en cours...")
    # Construction de l'URL PDB
    url = f'https://files.rcsb.org/download/{pdb_id}.pdb'

    try:
        # Télécharger le fichier PDB à partir de l'URL
        with urllib.request.urlopen(url) as response:
            pdb_content = response.read()

        # Définir le chemin local du fichier PDB
        pdb_file_path = f'{pdb_id.lower()}.pdb'

        # Écrire le contenu du fichier PDB dans le fichier local
        with open(pdb_file_path, 'wb') as pdb_file:
            pdb_file.write(pdb_content)
        print(f"Le fichier PDB de l'ID {pdb_id} a été téléchargé avec succès.")
        print("")
        return os.path.abspath(pdb_file_path)
    
    except urllib.error.HTTPError as e:
        print(f"Erreur lors de la récupération de la structure pour l'ID PDB {pdb_id}: {e}")
        return None

fichier1_pdb, fichier2_pdb = get_structure_from_pdb_id(pdb_id_1.value), get_structure_from_pdb_id(pdb_id_2.value)

Téléchargement du fichier PDB de l'ID 1ake en cours...
Le fichier PDB de l'ID 1ake a été téléchargé avec succès.

Téléchargement du fichier PDB de l'ID 4ake en cours...
Le fichier PDB de l'ID 4ake a été téléchargé avec succès.



Par chargement des fichiers :

In [6]:
pdb_file_1 = widgets.FileUpload(
    description = 'Charger le 1er fichier PDB',
    accept='.pdb',
    layout=widgets.Layout(width='220px'),
    multiple=False  
)

pdb_file_2 = widgets.FileUpload(
    description = 'Charger le 2nd fichier PDB',
    accept='.pdb',
    layout=widgets.Layout(width='220px'),
    multiple=False
)
# Fonction à appeler lors du clic sur le bouton
def visualisation_par_pdb(pdb_file_1, pdb_file_2):
    if isinstance(pdb_file_1, widgets.FileUpload) and isinstance(pdb_file_2, widgets.FileUpload):
        if pdb_file_1.value != () and pdb_file_2.value != ():
            #récupérer les noms des fichiers
            dic1, dic2 = pdb_file_1.value[0], pdb_file_2.value[0]
            nom1, nom2 = dic1['name'], dic2['name']
            # Visualiser avec nv
            view1 = nv.show_file(nom1)
            view2 = nv.show_file(nom2)
            display(view1)
            display(view2)
            return nom1, nom2

widgets.interact(visualisation_par_pdb,  pdb_file_1=pdb_file_1,  pdb_file_2=pdb_file_2);

interactive(children=(FileUpload(value={}, accept='.pdb', description='Charger le 1er fichier PDB', layout=Lay…

In [ ]:
fichier1_pdb, fichier2_pdb = visualisation_par_pdb(pdb_file_1, pdb_file_2)

In [57]:
# si le widget FileUpload ne fonctionne pas, on peut charger les fichiers manuellement
fichier1_pdb, fichier2_pdb = "1bta.pdb", "1brs_barstar_complexe.pdb"

In [ ]:
# visualisation des structures
visualisation_par_pdb("1bta.pdb", "1brs_barstar_complexe.pdb")

## III. Alignement des structures

In [7]:
# widget pour le choix des atomes de l'alignement
option_ali = widgets.ToggleButtons(
    options=['CA', 'backbone', 'all'],
    description='Alignement sur :',
    disabled=False,
    button_style='', 
    tooltips=['Alignement se fera sur les carbones alphas des 2 structures',
              'Alignement se fera sur les backbones des 2 structures',
              'Alignement se fera sur tous les atomes des 2 structures']
)
option_ali

ToggleButtons(description='Alignement sur :', options=('CA', 'backbone', 'all'), tooltips=('Alignement se fera…

In [8]:
def align_pdb_files(file1, file2, align_type):
    # Initialiser le parseur PDB
    parser = PDBParser()

    # Analyser les fichiers PDB et obtenir les structures
    structure1 = parser.get_structure('structure1', file1)
    structure2 = parser.get_structure('structure2', file2)

    # Accumuler les atomes pour l'alignement
    atoms1 = []
    atoms2 = []

    # Parcourir les résidus des deux structures
    for model1, model2 in zip(structure1, structure2):
        for chain1, chain2 in zip(model1, model2):
            for residue1, residue2 in zip(chain1, chain2):
                # Sélectionner les atomes en fonction du type d'alignement
                if align_type == "CA":
                    # Sélectionner les carbones alpha s'ils existent
                    if 'CA' in residue1 and 'CA' in residue2:
                        atoms1.append(residue1['CA'])
                        atoms2.append(residue2['CA'])
                elif align_type == "all":
                    # Sélectionner tous les atomes des résidus
                    atoms1.extend(residue1.get_atoms())
                    atoms2.extend(residue2.get_atoms())
                elif align_type == "backbone":
                    # Sélectionner les atomes du backbone (N, CA, C, O)
                    backbone_atoms1 = [atom for atom in residue1 if atom.id in ['N', 'CA', 'C', 'O']]
                    backbone_atoms2 = [atom for atom in residue2 if atom.id in ['N', 'CA', 'C', 'O']]
                    # Ajouter les atomes du backbone à la liste
                    atoms1.extend(backbone_atoms1)
                    atoms2.extend(backbone_atoms2)

    # Vérifier que les listes d'atomes ont la même longueur
    min_length = min(len(atoms1), len(atoms2))
    atoms1 = atoms1[:min_length]
    atoms2 = atoms2[:min_length]

    # Créer l'objet Superimposer et définir les atomes
    sup = Superimposer()
    sup.set_atoms(atoms1, atoms2)

    # Appliquer la transformation aux atomes de la structure 2
    sup.apply(structure2.get_atoms())

    return structure1, structure2

In [9]:
structure1, structure2 = align_pdb_files(fichier1_pdb, fichier2_pdb, option_ali.value)

# IV. Calcul du RMSD

In [10]:
# Fonction pour calculer le RMSD entre deux structures alignées
def calculate_rmsd(structure1, structure2, align_type=option_ali.value):
        atoms1 = []
        atoms2 = []

        for model1, model2 in zip(structure1, structure2):
            for chain1, chain2 in zip(model1, model2):
                for residue1, residue2 in zip(chain1, chain2):
                    if align_type == "CA":
                        try:
                            atoms1.append(residue1['CA'])
                            atoms2.append(residue2['CA'])
                        except KeyError:
                            pass
                    elif align_type == "all":
                        atoms1.extend(residue1.get_atoms())
                        atoms2.extend(residue2.get_atoms())
                    elif align_type == "backbone":
                        # Sélectionner les atomes du backbone (N, CA, C, O)
                        backbone_atoms1 = [atom for atom in residue1 if atom.id in ['N', 'CA', 'C', 'O']]
                        backbone_atoms2 = [atom for atom in residue2 if atom.id in ['N', 'CA', 'C', 'O']]
                        atoms1.extend(backbone_atoms1)
                        atoms2.extend(backbone_atoms2)

        total_atoms = len(atoms1)
        rmsd = sum([(atom1 - atom2) ** 2 for atom1, atom2 in zip(atoms1, atoms2)]) / total_atoms
        rmsd = rmsd ** 0.5

        return rmsd

In [14]:
rmsd = calculate_rmsd(structure1, structure2)
print(f"Le RMSD entre la structure {pdb_id_1.value} et {pdb_id_2.value} est égale à {rmsd:.4f} Å, en prenant en compte les atomes de type {option_ali.value}.")

Le RMSD entre la structure 1ake et 4ake est égale à 18.4487 Å, en prenant en compte les atomes de type CA.


## Petite Interface Graphique :)

In [17]:
# Définir une zone de sortie pour afficher le RMSD
output_rmsd = widgets.Output()

# Fonction pour calculer et afficher le RMSD
def calculate_and_display_rmsd(button):
    # Ici, vous calculeriez le RMSD entre les structures et stockeriez la valeur dans la variable rmsd
    rmsd = calculate_rmsd(structure1, structure2, option_ali.value)
    
    # Affichage formaté du RMSD
    with output_rmsd:
        # Effacer le contenu précédent
        output_rmsd.clear_output(wait=True)
        
        # Texte formaté avec les variables mises en valeur en couleur
        display(HTML(f"Le RMSD entre la structure <font color='blue'>{pdb_id_1.value}</font> et "
                     f"<font color='green'>{pdb_id_2.value}</font> est égale à "
                     f"<font color='red'>{rmsd:.4f} Å</font>, en prenant en compte les atomes de type "
                     f"<font color='purple'>{option_ali.value}</font>."))

        # Bouton pour copier la valeur du RMSD
        copy_button = widgets.Button(description="Copier RMSD")
        copy_button.on_click(lambda b: copy_to_clipboard(rmsd))
        display(copy_button)

# Fonction pour copier la valeur du RMSD dans le presse-papiers
def copy_to_clipboard(rmsd):
    import pyperclip
    pyperclip.copy(rmsd)
    print("La valeur du RMSD a été copiée dans le presse-papiers !")

# Créer un bouton pour déclencher le calcul du RMSD et l'affichage
calculate_button = widgets.Button(description="Calculer et Afficher RMSD", layout=widgets.Layout(width='220px'))
calculate_button.on_click(calculate_and_display_rmsd)

# Afficher les widgets et la zone de sortie
display(pdb_id_1, pdb_id_2, option_ali, calculate_button, output_rmsd)

Text(value='1ake', description='ID PDB 1:', placeholder='Entrez le premier ID PDB')

Text(value='4ake', description='ID PDB 2:', placeholder='Entrez le deuxième ID PDB')

ToggleButtons(description='Alignement sur :', options=('CA', 'backbone', 'all'), tooltips=('Alignement se fera…

Button(description='Calculer et Afficher RMSD', layout=Layout(width='220px'), style=ButtonStyle())

Output()